In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## A multigrid code ##

A simple code "example" of the multigrid relaxation method.
Solves $d^2\Phi/dx^2 = 0$ with $\Phi(0)=1$ and $\Phi(1)=0$.
The goal is to be clear, rather than efficient.

In [ ]:
# We discretize Phi"=0 on a regular, 1D grid [0,N] as
#       -Phi_{i-1}+2Phi_{i}-Phi_{i+1}=0
# where I have multiplied through by h^2 to eliminate the grid
# spacing (trivial since the rhs=0).
#
# At each step we solve the problem using the damped/weighted
# Jacobi/simultaneous displacement method:
#   Phi*_j = (1/2)[ Phi_{j-1}+Phi_{j+1} + h^2 F_j ]  (if source=F)
#   Phi_j  = 1/3 Phi_j + 2/3 Phi*_j
# where the 2/3 is the "optimal" weight.
#
# The multigrid steps are recursively implemented, with a "full multigrid"
# step calling a recursive V-cycle.

In [ ]:
def jacobi(phi,F,h,Niter=5,w=2./3.):
    """
    Does Niter iterations of the damped Jacobi method, with damping
    factor w.  The boundary conditions at the grid ends remain unchanged.
    """
    phistar=phi.copy()
    for iter in range(Niter):
        phistar[1:-1] = 0.5*(phi[:-2]+phi[2:]+h**2*F[1:-1])
        phi = (1-w)*phi + w*phistar
    return(phi)
    #

In [ ]:
def residual(phi,F,h):
    """Computes the residual."""
    d2phi = np.zeros_like(phi)
    d2phi[1:-1]=-phi[:-2]-phi[2:]+2*phi[1:-1]
    return(h**2*F-d2phi)
    #

In [ ]:
def prolong(v2h):
    """
    Transfer a vector, v2h, from the coarse grid to the fine grid (a
    factor of 2 finer) using linear interpolation.  This is often
    called interpolation or prolongation.
    In components v^h_{2j}=v^{2h}_j and v^h_{2j+1}=(1/2)[v^{2h}_j+v^{2h}_{j+1}]
    Note that I am keeping track of j=0 and N in the arrays, even though
    they are not "active".
    """
    v1h = np.empty( 2*v2h.size-1 )
    v1h[::2] = v2h[:]
    v1h[1::2]= 0.5*(v2h[:-1]+v2h[1:])
    v1h[0],v1h[-1]=v2h[0],v2h[-1]
    return(v1h)
    #

In [ ]:
def restrict(v1h):
    """
    Transfer a vector, v, from the fine grid to the coarse grid (a
    factor of 2 coarser) using full weighting.  This is often
    called restriction or injection.
    In components v^{2h}_{j}=(1/4)[v^{h}_{2j-1}+2v^h_{2j}+v^h_{2j+1}]
    Note that I am keeping track of j=0 and N in the arrays, even though
    they are not "active".
    """
    t1  = np.roll(v1h,-1)[::2]
    t2  = np.roll(v1h, 0)[::2]
    t3  = np.roll(v1h,+1)[::2]
    v2h = 0.25*(t1+2*t2+t3)
    v2h[0],v2h[-1]=v1h[0],v1h[-1]
    return(v2h)
    #

In [ ]:
def vcycle(phi,F,h,Niter=50):
    """
    Does one V-cycle with a recursive strategy--the Niter is passed
    through to the Jacobi method.
    """
    phi1 = jacobi(phi,F,h,Niter=Niter)
    F1   = F.copy()
    if len(phi1)>3:
        F2   = restrict(residual(phi1,F1,h))
        z2   = np.zeros_like(F2)
        # Note here the "F" is really a residual, so take out the
        # fact we really solve A.z=h^2.F
        v2   = vcycle(z2,F2/h**2,2*h,Niter=Niter)
        phi1+= prolong(v2)
    phi1 = jacobi(phi1,F1,h,Niter=Niter)
    return(phi1)
    #

In [ ]:
def fmg_vcycle(F,h,bndry,Niter=5):
    """
    Does one full multigrid V-cycle, using a recursive strategy.
    This implements nested iteration on top of the V-cycle correction scheme.
    """
    if len(F)>3:
        f2h = restrict(F)
        v2h = fmg_vcycle(f2h,2*h,bndry)
        v1h = prolong(v2h)
    else:
        # Start with zero, except for the boundary conditions.
        v1h = np.zeros_like(F)
        v1h[0],v1h[-1]=bndry[0],bndry[1]
    for iter in range(Niter):
        v1h = vcycle(v1h,F,h)
    return(v1h)
    #

In [ ]:
def solve(bndry=[1,0],n=16,Niter=50):
    """Try it out.  Note n should be a power of two."""
    # Set up the initial conditions and grids.
    xx      = np.linspace(0,1,n+1)
    F       = np.zeros(n+1)
    # Can either do a full multigrid or a bunch of V-cycles.
    if True:
        phi = fmg_vcycle(F,1.0/n,bndry)
    else:
        phi = np.zeros(n+1)
        phi[ 0],phi[-1] = bndry[0],bndry[1]
        for iter in range(5):
            phi = vcycle(phi,F,1.0/n,Niter=100)
    #
    exact = bndry[0] + (bndry[1]-bndry[0])*xx
    #
    print("# {:>4s} {:>10s} {:>10s} {:>10s}".\
          format("Iter","Approx","Exact","Error"))
    for i in range(n+1):
        print("{:6d} {:10.5f} {:10.5f} {:10.3f}".\
          format(i,phi[i],exact[i],100.*(phi[i]-exact[i])/(exact[i]+1e-10)))
    #

In [ ]:
solve()